<hr style="height:2px;">

# Download example data (do only once)

In [ ]:
import os, zipfile
from six.moves.urllib.request import urlretrieve
try:
    filepath = urlretrieve('https://cloud.mpi-cbg.de/index.php/s/z6VISEyibzgSFvY/download')[0]
    with zipfile.ZipFile(filepath,'r') as zipfile:
        zipfile.extractall('.')
finally:
    os.unlink(filepath)

Data should have been downloaded like this:

    raw_data/tribolium
    ├── GT
    │   └── nGFP_0.1_0.2_0.5_20_13_late.tif
    └── low
        └── nGFP_0.1_0.2_0.5_20_13_late.tif

<hr style="height:2px;">

# Training data generation for denoising of Tribolium

We demonstrate training data generation for the case where both low SNR and corresponding high SNR stacks can be acquired.  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
from tifffile import imread
from csbdeep import datagen
from csbdeep.plot_utils import plot_some

<hr style="height:2px;">

# Raw data

Corresponding low and high SNR stacks should have identical names but located in different folders. 
We only show one stack for demonstration, but you should have ~20-50 different stacks at different developmental timepoints.

Lets just plot the low and high (GT) stacks:

In [ ]:
y = imread('raw_data/tribolium/GT/nGFP_0.1_0.2_0.5_20_13_late.tif')
x = imread('raw_data/tribolium/low/nGFP_0.1_0.2_0.5_20_13_late.tif')
print('image size =', y.shape)

plt.figure(figsize=(15,10))
plot_some(np.stack([x,y]),title_list=[['low (maximum projection)','GT (maximum projection)']], pmin=2, pmax=99.8);

<hr style="height:2px;">

# Patch generation

From corresponding stacks, we generate 3D patches of size (16,128,128) that are sampled from regions of interesting structures. 

In [ ]:
input_data = datagen.get_tiff_pairs_from_folders (
    basepath   = 'raw_data/tribolium',
    input_dirs = ['low'],
    output_dir = 'GT'
)

X, Y = datagen.create_patches (
    input_data = input_data,
    patch_size = (16,128,128),
    n_samples  = 32
)

In [ ]:
print("shape of X", X.shape)
print("shape of Y", Y.shape)

This shows the maximum projection of some of those patch pairs (odd rows are inputs, even rows outputs)

In [ ]:
from csbdeep.plot_utils import plot_some

for i in range(4):
    plt.figure(figsize=(16,4))
    sl = slice(8*i, 8*(i+1))
    plot_some(X[sl],Y[sl])
    plt.show()
None;